In [14]:
import numpy as np
from sklearn.neighbors import DistanceMetric

In [15]:
CFG={
    'dist_metric':'euclidean',
    'pivots_file_path':'/home/kvassay/Documents/data/sketch-testing/pivots-random-320.data',
    'base_sketch_length':32,
    'dataset_path':'/home/kvassay/Documents/data/sketch-testing/profi-neuralnet-100K.data'
    'desired_sketch_length': 16
}

SyntaxError: invalid syntax (<ipython-input-15-ec5d5a3343d1>, line 9)

In [16]:
def get_cfg(cfg_path_or_dict):
    if type(cfg_path_or_dict) == dict:
        return cfg_path_or_dict
    if not os.path.isfile(cfg_path_or_dict):
        raise ValueError('Config must be path to valid .yml file or dict.')
    with open(cfg_path_or_dict, 'rb') as f:
        cfg = yaml.load(f)
    return cfg

In [17]:
class CaffeVectorsIterator():
    def __init__(self, file_path):
        self.file = open(file_path, 'rb')

    def next(self):
        try:
            # (id: int, vector: tuple of float)
            lineA = self.file.next()
            lineB = self.file.next()
            print(lineA)
            print(lineB)
            return (int(lineA.split(' ')[2]), np.array([float(num) for num in lineB.split(' ')]))
        except StopIteration:
            self.file.close()
            raise StopIteration

    def __del__(self):
        self.file.close()


class CaffeVectorsIterable():
    def __init__(self, file_path):
        self.file_path = file_path

    def __iter__(self):
        return CaffeVectorsIterator(self.file_path)

In [18]:
class MultiHyperplanePartitioner():
    #FIX! !!!! TAKE ALL PIVOTS, COMPUTE 50/50 ratio and take k top
    def __init__(self, cfg):
        cfg = get_cfg(cfg)
        self.dist_metric = DistanceMetric(cfg['dist_metric'])  # for example 'euclidean'
        self.read_pivots(cfg)
    
    def read_pivots(self, cfg):
        self.pivots = list()
        iterator = CaffeVectorsIterable(cfg['pivots_file_path']).__iter__()

        while True:
            try:
                pivot_A = iterator.next()
                pivot_B = iterator.next()
                self.pivots.append((pivot_A, pivot_B))
            except StopIteration:
                break
        
            
    def get_partition(self, pivot_couple, obj):
        distance_A = self.dist_metric.pairwise(obj[1], pivot_couple[0][1])[0][0]  # pairwise returns matrix
        distance_B = self.dist_metric.pairwise(obj[1], pivot_couple[1][1])[0][0]
        if distance_A > distance_B:
            return 0
        else:
            return 1

    def get_partitions(self, obj, index_mask=None):
        if not index_mask:
            index_mask=[True]*len(self.pivots)
        return [self.get_partition(pivot_couple, obj) for index, pivot_couple in enumerate(self.pivots) if index_mask[index]]

In [20]:
class SketchTrasformator():
def __init__(self, cfg):
    self.cfg=self.get_cfg(cfg)
    self.partitioner=MultiHyperplanePartitioner(cfg)
    dataset_iterator=CaffeVectorsIterable(cfg['dataset_path']).__iter__()
    #sketch_matrix=np.matrix([self.partitioner.get_partitions(data_record) for data_record in dataset_iterator])
    self.index_mask=generate_parition_index_mask(sketch_matrix, int(cfg['desired_sketch_length']))
    
def get_sketches(length):
    assert length<=len(self.partitioner.pivots)

def generate_partition_index_mask(sketch_matrix, desired_sketch_len):
    #if not enough pivots 
    pass
    
        

IndentationError: expected an indented block (<ipython-input-20-c49506883cab>, line 14)